# Step 1: Importing Libraries

In [84]:
import pandas as pd
import numpy as np
import re  
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping

# Loading dataset

In [51]:
data = pd.read_csv('swiggy.csv')
print("Columns in the dataset:")

Columns in the dataset:


In [52]:
data.head()

,ID,Area,City,Restaurant Price,Avg Rating,Total Rating,Food Item,Food Type,Delivery Time,Review
0,1,Suburb,Ahmedabad,600,4.2,6198,Sushi,Fast Food,30-40 min,"Good, but nothing extraordinary."
1,2,Business District,Pune,200,4.7,4865,Pepperoni Pizza,Non-Vegetarian,50-60 min,"Good, but nothing extraordinary."
2,3,Suburb,Bangalore,600,4.7,2095,Waffles,Fast Food,50-60 min,Late delivery ruined it.
3,4,Business District,Mumbai,900,4.0,6639,Sushi,Vegetarian,50-60 min,Best meal I've had in a while!
4,5,Tech Park,Mumbai,200,4.7,6926,Spring Rolls,Gluten-Free,20-30 min,Mediocre experience.


In [53]:
print(data.columns.tolist())

['ID', 'Area', 'City', 'Restaurant Price', 'Avg Rating', 'Total Rating', 'Food Item', 'Food Type', 'Delivery Time', 'Review']


# Preprocessing

In [58]:
data["Review"] = data["Review"].str.lower()
data["Review"] = data["Review"].replace(r'[^a-z0-9\s]', '', regex=True)

In [64]:
data['sentiment'] = data['Avg Rating'].apply(lambda x: 1 if x > 3.5 else 0)

In [65]:
data = data.dropna() 

# Tokenization and Padding

In [66]:
max_features = 5000  
max_length = 200    

In [67]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(data["Review"])
X = pad_sequences(tokenizer.texts_to_sequences(data["Review"]), maxlen=max_length)
y = data['sentiment'].values

# Data Splitting

In [68]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=42, stratify=y_train
)

# Define the model

In [70]:
model = Sequential([
    Embedding(input_dim=max_features, output_dim=64),
    SimpleRNN(64, activation='tanh', return_sequences=False),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Compile the mode

In [71]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Train the model

In [72]:
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_val, y_val),
    callbacks=[early_stop],
    shuffle=True,
    verbose=1
)

Epoch 1/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.6758 - loss: 0.6297 - val_accuracy: 0.7156 - val_loss: 0.6022
Epoch 2/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 8s 46ms/step - accuracy: 0.7233 - loss: 0.5998 - val_accuracy: 0.7156 - val_loss: 0.5972
Epoch 3/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - accuracy: 0.7174 - loss: 0.5980 - val_accuracy: 0.7156 - val_loss: 0.5957
Epoch 4/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms/step - accuracy: 0.6880 - loss: 0.6271 - val_accuracy: 0.7156 - val_loss: 0.6172
Epoch 5/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 7s 36ms/step - accuracy: 0.6379 - loss: 0.6659 - val_accuracy: 0.7156 - val_loss: 0.5980


# Evaluate the Model

In [73]:
score = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {score[1]:.2f}")

Test accuracy: 0.72


# Predicting sentiment

In [74]:
def predict_sentiment(review_text):
    text = review_text.lower()
    text = re.sub(r'[^a-z0-9\s]', '', text)

    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length)

    prediction = model.predict(padded)[0][0]
    return f"{'Positive' if prediction >= 0.5 else 'Negative'} (Probability: {prediction:.2f})"

# Sample usage

In [75]:
sample_review = "The food was great."
print(f"Review: {sample_review}")
print(f"Sentiment: {predict_sentiment(sample_review)}")

Review: The food was great.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 607ms/step
Sentiment: Positive (Probability: 0.66)
